<h3>Unificar n base de datos</h3>
<item>
<li> Las entradas todas guardan la misma estructura
<li> Las fuentes tienen el mismo sistema de referencia
</item>

<h4>Configurando Librerías</h4>

In [ ]:
import os, shutil, arcpy
from pathlib import Path

<h4>Configuración de variables</h4>

In [ ]:
ruta_analisis = r"C:\Users\rodian.saby\OneDrive - Valor + SAS\2024\3.Zonas_Homogeneas_Fisicas\0.PRO\VMas_ZHG\0.Insumos\CatastroAntioquia_Enero\CONTRALORIA"
nombre_directorio = "backUP_GDB"
nombre_nueva_basedatos = "ZHF_Unificada.gdb"
bd_para_esquema = "M002.gdb"
bd_muestra_xml = os.path.join(ruta_analisis, nombre_directorio, bd_para_esquema) 

<h4>Creación de carpeta donde se almacenaran las BD que sirven de BackUP</h4>

In [ ]:

# ** Esta carpeta se crea como subdirectorio de la carpeta raiz (ruta_analisis)

if os.path.exists(os.path.join(ruta_analisis, nombre_directorio)):
    print("El directorio {0} existe".format(nombre_directorio))
else:
    os.mkdir(os.path.join(ruta_analisis, nombre_directorio))
    print("Se crea el directorio {0}".format(nombre_directorio))

<h4>Se copian todas las bases de datos (.gdb) a la carpeta creada</h4>

In [ ]:
listado_directorios = os.listdir(ruta_analisis)

directorios_analisis = []
for directorio in listado_directorios:
    if directorio.endswith(".zip"):
      pass
    else:
        directorios_analisis.append(os.path.join(ruta_analisis,directorio))

for i in directorios_analisis:
   for ruta, directorio, archivo in os.walk(i, topdown=True):
      if ruta.endswith(".gdb"):
         p = Path(ruta)
         directorio_destino = os.path.join(ruta_analisis, nombre_directorio, p.name)
         if os.path.exists(directorio_destino):
            print("El archivo {0} existe".format(p.name))
         else:            
            shutil.copytree(ruta, directorio_destino)
            print("Se copia {0}".format(ruta), '\n' "Al destino {0}".format(directorio_destino))

<h4>Incorporación de campo dentro de cada BD</h4>
<p>Se adicionará un campo y se registrará el nombre de la base de datos</p>

In [ ]:
lista_basesdatos = os.listdir(os.path.join(ruta_analisis, nombre_directorio))

rutas_bd_espacio_trabajo = []
for i in lista_basesdatos:
    ruta_bd = os.path.join(ruta_analisis, nombre_directorio, i)
    p = Path(ruta_bd)
    if ruta_bd.endswith(".gdb"):
        rutas_bd_espacio_trabajo.append(ruta_bd)        
    else:
        pass

for j in rutas_bd_espacio_trabajo:
    p = Path(j)
    nombre_municipio = p.stem.replace('M','05')
    arcpy.env.workspace = j
    for dataset in arcpy.ListDatasets():
        for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
            arcpy.management.AddField(capa, 'codigo_municipio', field_type = 'TEXT', field_length = 5, field_alias = 'Código Municipio')
            arcpy.management.CalculateField(capa, 'codigo_municipio','"'+nombre_municipio+'"', expression_type = 'PYTHON3')
            print("Se actualiza la capa {0}, dataframe {1} de la BD {2}".format(capa, dataset, p.name))

<h4>Generación de XML sobre la BD de muestra</h4>

In [ ]:
ruta_nueva_bd = os.path.join(ruta_analisis, nombre_directorio)

p = Path(bd_muestra_xml)
nombre_xml_referencia = p.name.split('.')[0] + ".xml"

if os.path.exists(os.path.join(ruta_nueva_bd, nombre_xml_referencia)):
    print("El XML {0} existe".format(nombre_xml_referencia))
else:
    arcpy.ExportXMLWorkspaceDocument_management(bd_muestra_xml, os.path.join(ruta_nueva_bd, nombre_xml_referencia), export_type = 'SCHEMA_ONLY')
    print("Se genera XML {0}".format(nombre_xml_referencia))

<h4>Creación de BD para la unificación y importación de XML</h4>

In [ ]:
if os.path.exists(os.path.join(ruta_nueva_bd, nombre_nueva_basedatos)):
    print("La base de datos {0} existe".format(nombre_nueva_basedatos))
else: 
    arcpy.CreateFileGDB_management(ruta_nueva_bd, nombre_nueva_basedatos)

    arcpy.management.ImportXMLWorkspaceDocument(os.path.join(ruta_analisis, nombre_directorio, nombre_nueva_basedatos), 
                                                os.path.join(ruta_nueva_bd, nombre_xml_referencia), 
                                                import_type = "SCHEMA_ONLY")
    print("Se importa XML {0}".format(nombre_xml_referencia))

<h4>Unificación de BD</h4>

In [ ]:
lista_basesdatos = os.listdir(os.path.join(ruta_analisis, nombre_directorio))

rutas_bd_espacio_trabajo = []
for i in lista_basesdatos:
    ruta_bd = os.path.join(ruta_analisis, nombre_directorio, i)
    p = Path(ruta_bd)
    if ruta_bd.endswith(".gdb") and p.name != nombre_nueva_basedatos:
        rutas_bd_espacio_trabajo.append(ruta_bd)        
    else:
        pass

for base in rutas_bd_espacio_trabajo:
    arcpy.env.workspace = base
    for dataset in arcpy.ListDatasets():
        for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
            entrada = os.path.join(base, dataset, capa)
            p = Path(entrada)
            salida = os.path.join(str(ruta_analisis), str(nombre_directorio), str(nombre_nueva_basedatos), str(dataset), str(capa))
            arcpy.management.Append(inputs = entrada, target = salida, schema_type = 'NO_TEST')
            print("Se unifica {0}, dataset {1} de la base de datos origen {2}".format(capa, dataset, p.parts[11]))